In [100]:
import pandas as pd
from sqlalchemy import create_engine

### Extract CSVs into DataFrames

In [101]:
cyclist_file = "../ETL project/cyclist.csv"
cyclist_df = pd.read_csv(cyclist_file)
cyclist_df

,Date,Brooklyn Bridge
0,10/01/17,2297
1,10/02/17,3387
2,10/03/17,3386
3,10/04/17,3412
4,10/05/17,3312
5,10/06/17,2982
6,10/07/17,2750
7,10/08/17,1235
8,10/09/17,898
9,10/10/17,3922


In [103]:
pedestrian_file = ("../ETL project/pedestrian_counts.csv")
pedestrian_db = pd.read_csv(pedestrian_file)
pedestrian_db

,hour_beginning,location,Pedestrians,Towards Manhattan,Towards Brooklyn,weather_summary,temperature,precipitation,lat,long,events
0,10/01/17,Brooklyn Bridge,44,30,14,clear-night,52,0.0001,40.708164,-73.999509,NaN
1,10/01/17,Brooklyn Bridge,30,17,13,partly-cloudy-night,53,0.0002,40.708164,-73.999509,NaN
2,10/01/17,Brooklyn Bridge,25,13,12,partly-cloudy-night,52,0.0000,40.708164,-73.999509,NaN
3,10/01/17,Brooklyn Bridge,20,11,9,partly-cloudy-night,51,0.0000,40.708164,-73.999509,NaN
4,10/01/17,Brooklyn Bridge,18,10,8,partly-cloudy-night,51,0.0000,40.708164,-73.999509,NaN
5,10/01/17,Brooklyn Bridge,16,12,4,partly-cloudy-night,50,0.0000,40.708164,-73.999509,NaN
6,10/01/17,Brooklyn Bridge,113,88,25,partly-cloudy-night,49,0.0001,40.708164,-73.999509,NaN
7,10/01/17,Brooklyn Bridge,199,72,127,clear-day,49,0.0000,40.708164,-73.999509,NaN
8,10/01/17,Brooklyn Bridge,261,133,128,clear-day,50,0.0000,40.708164,-73.999509,NaN
9,10/01/17,Brooklyn Bridge,845,504,341,clear-day,54,0.0000,40.708164,-73.999509,NaN


### Transform premise DataFrame

In [104]:
cyclist = ["Date", "Brooklyn Bridge"] 
cyclist_transform = cyclist_df[cyclist].copy()

cyclist_transform = cyclist_transform.rename(columns={"Date": "date",
                                                      "Brooklyn Bridge": "number_of_cyclists",
                                                          })
cyclist_transform.set_index("date", inplace=True)
cyclist_transform.head()

,number_of_cyclists
date,
10/01/17,2297
10/02/17,3387
10/03/17,3386
10/04/17,3412
10/05/17,3312


### Transform county DataFrame

In [105]:
pedestrian = ["hour_beginning", "Pedestrians"]
pedestrian_transform = pedestrian_db[pedestrian].copy()

pedestrian_transform = pedestrian_transform.rename(columns = {"hour_beginning": "date", 
                                                     "Pedestrians": "number_of_pedestrians",
                                                     })

pedestrian_transform.drop_duplicates("date", inplace=True)
pedestrian_transform.set_index("date", inplace=True)
pedestrian_transform.head()

,number_of_pedestrians
date,
10/01/17,44
10/02/17,60
10/03/17,45
10/04/17,28
10/05/17,41


### Create database connection

In [106]:
connection = "postgres:Franklin#4@localhost:5432/brooklyn"
engine = create_engine(f'postgresql://{connection}')

In [107]:
# Confirm tables
engine.table_names()

['cyclist', 'pedestrian']

### Load DataFrames into database

In [108]:
cyclist_transform.to_sql(name='cyclist', con=engine, if_exists='append', index=True)
pedestrian_transform.to_sql(name='pedestrian', con=engine, if_exists='append', index=True)